In [ ]:
import os
import torch
import numpy as np
import torch.nn as nn
import pandas as pd

In [ ]:
PM10 = pd.read_csv('./data/PM10-2023.csv', sep=';')
NO2 = pd.read_csv('./data/NO2-2023.csv', sep=';')
traffic = pd.DataFrame()
for file in os.listdir('./data/natezenie_ruchu_2023'):
    data = pd.read_csv('./data/natezenie_ruchu_2023/' + file, sep=';')
    traffic = pd.concat([traffic, data])

In [ ]:
PM10 = PM10.drop_duplicates()
traffic = traffic.drop_duplicates()
NO2 = NO2.drop_duplicates()

In [ ]:
traffic.columns = ['Traffic_station_id', 'date', 'hour', 'quality', 'num_vehicles', 'avg_speed', 'num_cars', 'avg_car_speed', 'num_trucks', 'avg_truck_speed']
df = pd.read_excel('./data/Stammdaten_Verkehrsdetektion_2022_07_20.xlsx')
# a = []
# for i, item in enumerate(df['DET_ID15']):
#     if item in traffic['station_id'].unique():
#         a.append([df['x'][i], df['y'][i]])

In [ ]:
traffic_id = [100101010031590, 100101010031287]
traffic['date'] = pd.to_datetime(traffic['date'], format="%d.%m.%Y")
traffic['hour'] = pd.to_timedelta(traffic['hour'], unit='h')
traffic['datetime'] = traffic['date'] + traffic['hour']
traffic = traffic.drop(columns=['date', 'hour'])
traffic = traffic[traffic['Traffic_station_id'].isin(traffic_id)]

In [ ]:
PM10['Date'] = pd.to_datetime(PM10['Date'], format="%Y-%m-%d")
PM10['Date'][PM10['Time'] == '24:00'] = PM10['Date'] + pd.Timedelta(days=1)
PM10['Time'][PM10['Time'] == '24:00'] = '00:00'
PM10['Time'] = pd.to_timedelta(PM10['Time'].str.strip("'") + ':00')
PM10['datetime'] = PM10['Date'] + PM10['Time']
PM10 = PM10.drop(columns=['Date', 'Time', "State / Measuring network", "Station setting","Station type","Pollutant","Time scope", "Unit", "Station name"])

In [ ]:
print(len(traffic))
print(len(PM10))

In [ ]:
merged_df = pd.merge(traffic, PM10, left_on='datetime', right_on='datetime', how='inner')
pivot_df = merged_df.pivot_table(index=['datetime'],
                          columns=['Station code', 'Traffic_station_id'],
                          values=['num_vehicles', 'avg_speed', 'num_cars', 'avg_car_speed',
                                  'num_trucks', 'avg_truck_speed', 'Measure value','quality'],
                          aggfunc='first')
pivot_df.columns = ['{}_{}_{}'.format(col[1], col[0], i) for i, col in enumerate(pivot_df.columns.values)]
pivot_df.reset_index(inplace=True)

In [ ]:
columns_to_drop = [1, 3, 5, 6, 8, 9, 11, 12, 14, 15, 17, 18, 20, 21, 23, 24, 26, 27, 29, 30]
pivot_df = pivot_df.drop(pivot_df.columns[columns_to_drop], axis=1)
pivot_df.to_csv("pivot.csv", index=False)
merged_df.to_csv("merged.csv", index=False)